In [21]:
!pip3  install pyspark
!pip3 install findspark
!pip3 install wordcloud
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install sklearn
!pip3 install pandas
!pip3 install numpy

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SK

In [22]:
!pip3 install scikit-learn

In [23]:
# ====================================
# 🔧 SETUP & IMPORTS
# ====================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, concat_ws, udf
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, MinHashLSH
)
from pyspark.ml.linalg import Vectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from wordcloud import WordCloud

In [24]:
# Create the .kaggle directory and move the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set permissions for the json file
!chmod 600 ~/.kaggle/kaggle.json

mv: kaggle.json: No such file or directory


In [25]:
!pip3 install kagglehub

import os
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

/Users/moeinghaeini/Desktop/Amazon Book Review/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
import os
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# List of CSV files in the dataset
file_names = [
    "books_data.csv", "Books_rating.csv"
]

# Directory to save CSVs locally
save_dir = "dataset/full_dataset"
os.makedirs(save_dir, exist_ok=True)

# Loop through all files, load and save locally
for file_name in file_names:
    print(f"Loading {file_name} ...")
    df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "mohamedbakhet/amazon-books-reviews",
        file_name,
    )
    save_path = os.path.join(save_dir, file_name)
    df.to_csv(save_path, index=False)
    print(f"Saved {file_name} to {save_path}")

/var/folders/yc/m8cs0qcn1lg94kgtm6j1s0th0000gn/T/ipykernel_29606/3121204917.py:18: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Loading books_data.csv ...
Saved books_data.csv to dataset/full_dataset/books_data.csv
Loading Books_rating.csv ...


/var/folders/yc/m8cs0qcn1lg94kgtm6j1s0th0000gn/T/ipykernel_29606/3121204917.py:18: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Saved Books_rating.csv to dataset/full_dataset/Books_rating.csv


In [27]:
# --- SparkContext setup
# ====================================
# 🚀 INITIATE SPARK
# ====================================
spark = SparkSession.builder.appName("SimilarBooksAcademic").getOrCreate()

In [28]:
books = spark.read.csv("dataset/full_dataset/books_data.csv", header=True, inferSchema=True)
ratings = spark.read.csv("dataset/full_dataset/Books_rating.csv", header=True, inferSchema=True)

In [29]:
# prompt: print schema

books.printSchema()
ratings.printSchema()

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- image: string (nullable = true)
 |-- previewLink: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- infoLink: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [30]:
!pip3 install nltk

In [31]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, MinHashLSH
import nltk

In [32]:

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/moeinghaeini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moeinghaeini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/moeinghaeini/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
!pip3 install pyspark

In [34]:
import pandas as pd
import os
import numpy as np

# Create a new directory for sampled data
if not os.path.exists('sampled_data'):
    os.makedirs('sampled_data')

# Load the full dataset
print("Loading full dataset...")
df = pd.read_csv('dataset/full_dataset/Books_rating.csv')

# Sample 10% of the data with random_state for reproducibility
print("Sampling 10% of the data...")
sampled_df = df.sample(frac=0.1, random_state=42)

# Save the sampled data
print("Saving sampled data...")
sampled_df.to_csv('sampled_data/Books_rating_sampled.csv', index=False)

# Print statistics
print(f"Original dataset size: {len(df)} reviews")
print(f"Sampled dataset size: {len(sampled_df)} reviews")
print(f"Sampled data saved to: sampled_data/Books_rating_sampled.csv")

Loading full dataset...
Sampling 10% of the data...
Saving sampled data...
Original dataset size: 3000000 reviews
Sampled dataset size: 300000 reviews
Sampled data saved to: sampled_data/Books_rating_sampled.csv


In [35]:
import pandas as pd
import os
import numpy as np

# Create a new directory for sampled data
if not os.path.exists('sampled_data'):
    os.makedirs('sampled_data')

# Load the full dataset
print("Loading full dataset...")
df = pd.read_csv('dataset/full_dataset/books_data.csv')

# Sample 10% of the data with random_state for reproducibility
print("Sampling 10% of the data...")
sampled_df = df.sample(frac=0.1, random_state=42)

# Save the sampled data
print("Saving sampled data...")
sampled_df.to_csv('sampled_data/books_data_sampled.csv', index=False)

# Print statistics
print(f"Original dataset size: {len(df)} reviews")
print(f"Sampled dataset size: {len(sampled_df)} reviews")
print(f"Sampled data saved to: sampled_data/books_data_sampled.csv")

Loading full dataset...
Sampling 10% of the data...
Saving sampled data...
Original dataset size: 212404 reviews
Sampled dataset size: 21240 reviews
Sampled data saved to: sampled_data/books_data_sampled.csv
